In [176]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

print('Libraries Imported.')

Libraries Imported.


In [177]:
# launch webdriver of target and gather html
url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

options = webdriver.ChromeOptions() 
# options.add_argument('--headless') # run driver without browser
driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                            options=options)
driver.get(url)
# driver.implicitly_wait(10)

In [178]:
# navigate to product line tab
frameset = driver.find_element_by_tag_name('frameset')
print("Frameset: ", frameset)
frame = frameset.find_element_by_tag_name('frame')
frame_name = frame.get_attribute('name')
print(frame_name)
# grab last 6 chars of frame name since it changes everytime page is loaded
frame_suffix = frame_name[11:17]
print(frame_suffix)

# set driver in frame that contains necessary info
driver.switch_to.frame(frame)
# find product line tab
product_line = driver.find_element_by_id("Tab2")

# wait for product line tab to load before clicking
# wait = WebDriverWait(browser, 10)
# productList = wait.until(EC.presence_of_element_located((By.XPATH, '//html/body/table/tbody/tr[2]/td/div[2]/ul')))

# select product line tab
product_line.click()
# print(browser.page_source)

Frameset:  <selenium.webdriver.remote.webelement.WebElement (session="4defcdca717bb3c2c25f2fbbb9043fb6", element="c766f17c-d851-4ea2-8fbe-b4804ea2f112")>
mainWindow_opKUaa
opKUaa
Frame:  https://echoinc.partsmartweb.com/scripts/EmpartISAPI.dll?MF&app=echo&session=8820e56e-40ad-4d21-8629-28904e2bb4f7&mode=12&TF=ewMain


In [179]:
# set names/IDs of frames based on 6 chars that were previously saved
prodLineFrameID = 'iframeSearchProdLine_' + frame_suffix
detailFrameID = 'plDetailFrame'
dataFrameName = 'ARICAT6TREE_' + frame_suffix

# navigate into frame that contains jstree of all products/parts
frame = driver.find_element_by_id(prodLineFrameID)
driver.switch_to.frame(frame)

frame = driver.find_element_by_id(detailFrameID)
driver.switch_to.frame(frame)

frame = driver.find_element_by_name(dataFrameName)
driver.switch_to.frame(frame)

# click into ECHO jstree
cur_node = driver.find_element_by_id('pn18')
cur_node.click()

In [180]:
def getNodes(nodeList):
    # get all list items from unordered list param
    nodes = nodeList.find_elements_by_tag_name("li")

    # **on first run getParts is correctly gathering all li of first ul**
    # for node in nodes:
    #     print(node.get_attribute('id'))

    # iterate through all list items in current unordered list
    for node in nodes:
        # grab and store node ID
        node_id = node.get_attribute('id')
        # print ID of node
        print("List item ID: ", node_id)
        # grab and store ID of corresponding anchor
        anchor_id = node_id + '_anchor'
        # print anchor ID
        print('Anchor ID: ', anchor_id)

        # click the anchor
        # driver.find_element_by_id(anchor_id).click()

        # print(node.get_attribute('outerHTML'))

        # make driver wait for node to be clickable before clicking
        wait = WebDriverWait(driver, 10)
        anchor = wait.until(EC.element_to_be_clickable((By.ID, anchor_id))).click()

        # regather node web element now that reference node has been clicked (trying to avoid stale element exception)
        node = driver.find_element_by_id(node_id)

        # (base case) check if node is a parent, if false gather info from leaf node
        if node.get_attribute('data-assembly-is-parent') == "false":
            # TO DO: gather data from part and pull to dataframe?
            print(' Reached leaf node.')
        # else, click the element to expand child list
        # else:          
            # print(node.get_attribute('innerHTML'))
           
            # tried explicit waits to avoid stale element references and issues of driver clicking random nodes
            # ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
            # next_ul = WebDriverWait(driver, 30, ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.TAG_NAME, 'ul')))
            # getNodes(next_ul)
            
            # print(node.get_attribute('outerHTML'))
            
            # find next unordered list element and call getParts recursively to iterate through next level
            # getNodes(node.find_element_by_tag_name("ul"))

In [181]:
# grab first unordered list in ECHO file
# productList = driver.find_element_by_tag_name("ul")

wait = WebDriverWait(driver, 10)
productList = wait.until(EC.element_to_be_clickable((By.TAG_NAME, 'ul')))

# pass to getParts to recursively open all nodes
getNodes(productList)

List item ID:  j1_loading
Anchor ID:  j1_loading_anchor


TimeoutException: Message: 


In [ ]:
# opens all nodes of jstree, but it takes a very long time and not sure how to tell when it's done
driver.execute_script("return $('.jstree').jstree('open_all');")

In [ ]:
# open a new webdriver for each product group under ECHO file to control run time and resources
